# Graphs in ML - Project Notebook
###### Valentin Berkes, Simon Lebastard

In this notebook we will create several strongly and weakly connected graphs, test the Exp3G algorithm and assess the influence of a qualitative change in the connectivity graph on the evolution of regret.

In [ ]:
import networkx as nx
import pygraphviz
from networkx.drawing.nx_agraph import write_dot
from networkx.algorithms.approximation import *
from EXP3 import EXP3, EXP3Opt, compute_regret, plot_regret, gaussian_filter, upper_bound
import arms
import numpy as np
import obsGraph
import pdb

In [ ]:
G = nx.DiGraph()
G.add_node(0, arm=arms.ArmBernoulli(0.5))
G.add_node(1, arm=arms.ArmBernoulli(0.3))
G.add_node(2, arm=arms.ArmBernoulli(0.4))
G.add_node(3, arm=arms.ArmBernoulli(0.7))
G.add_edge(0,0)
G.add_edge(0,1)
G.add_edge(2,2)
G.add_edge(3,3)

G = nx.convert_node_labels_to_integers(G)

In [ ]:
import matplotlib.pyplot as plt
nx.draw(G)  # networkx draw()
plt.draw()
plt.show()

In [ ]:
## Get nice graphs with self-loops in PNG format
# 1) Install pygraphviz
# 2) Run:
write_dot(G,'graph.dot')
# 3) In terminal, run: dot -Tpng graph.dot > graph.png

We will run the Exp3G algorithm 50 times and produce an average to have a smooth regret function. This will allow us to detect quasi-linear components and better identify the asymptotic regret. Note that quasi-linear components can be used on a transformed regret:
- $x \mapsto \sqrt{x}$ would allow us to find the areas where the regret behaves as $\mathcal{O}(\sqrt{x})$
- $x \mapsto x^{\frac{2}{3}}$ would allow us to find the areas where the regret behaves as $\mathcal{O}(x^{\frac{2}{3}})$

In [ ]:
n_itr = 100000
n_sim = 50

# q,losses = EXP3(G, list(G.nodes()), 0.5, 0.05, n_itr, n_sim)
q,losses = EXP3Opt(G, list(G.nodes()), n_itr, n_sim)

In [ ]:
regret = compute_regret(losses, G)

Fitting power functions is still experimental, but it will allow us to track changes in regret trends when it fully works.
Note that we could better fit by knowing the independence number $\alpha(G)$ for strongly connected graphs, or the weak domination number $\delta(G)$ for weakly connected graphs. Determining those values is NP-hard, so no scalable method will be available, but there are some algorithms for computing approximations for small graphs. See for instance (Fox & Pach)

In [ ]:
der2,linAreas = plot_regret(G, [regret, upper_bound(G, len(regret))], ['EXP3', 'Upper bound'])#, reg="Pwr2/3", stdev=34)
# der2,linAreas = plot_regret([regret], ['EXP3'])

Second derivative can be plotted to figure out the thresholds to use for fitting linear and power curves

In [ ]:
#plt.plot(range(10,4700), der2[10:4700])
#plt.show()

## Building strongly connected graphs

To generate a class of strongly connected graphs, we use a parametric method that proceeds as follows:
- A fully connected graph $\mathcal{G}$ is created
- $\alpha \in [0,1]$ specifies the rate of self-edges to be removed from $\mathcal{G}$
- $\beta \in [0,1]$ parametrises the rate of peer-edges to be removed, according to the following policy: if we decide to remove peer-edges for a node $i$ then there is a uniform probability distribution over the number of peer-edges to remove.

Even though this method does allow to generate only a given class of graphs, we can later generalize it by introducing a third parameter $p$ that would be the probability distribution to replace the uniform distribution in the case where peer-edges are removed.

Here is an example of a strongly connected graph created through our function:

In [ ]:
alpha1 = 0.5
beta1 = 0.5
n_nodes = 5
H1 = obsGraph.strong_obs_graph(n_nodes, alpha1, beta1)

In [ ]:
nx.draw(H1)  # networkx draw()
plt.draw()
plt.show()
write_dot(H1,'strong1.dot')

In [ ]:
obsGraph.observability_type(H1)

In [ ]:
obsGraph.strong_nodes(H1)

Note that in this case, we chose $\alpha + \beta = 1$, resulting in edges removed for all nodes (either self-edge, or one or more peer-edges).

In the second example below, some edges are left dual:

In [ ]:
alpha2 = 0.4
beta2 = 0.3
n_nodes = 8
H2 = obsGraph.strong_obs_graph(n_nodes, alpha2, beta2)
nx.draw(H2)  # networkx draw()
plt.draw()
plt.show()
#write_dot(H2,'strong2.dot')
obsGraph.observability_type(H2)
obsGraph.strong_nodes(H2)

# Examples
## Strongly observable
### Bandit

In [ ]:
bandit = nx.DiGraph()
bandit.add_node(0, arm=arms.ArmBernoulli(0.5))
bandit.add_node(1, arm=arms.ArmBernoulli(0.3))
bandit.add_node(2, arm=arms.ArmBernoulli(0.4))
bandit.add_node(3, arm=arms.ArmBernoulli(0.7))
bandit.add_node(4, arm=arms.ArmBernoulli(0.1))
bandit.add_edge(0,0)
bandit.add_edge(1,1)
bandit.add_edge(2,2)
bandit.add_edge(3,3)
bandit.add_edge(4,4)
print(obsGraph.observability_type(bandit))

In [ ]:
nx.draw(bandit)
plt.draw()
plt.show()
# Note that networkx does not display self edges
# Exporting the dot graph allows to see self-edges

In [ ]:
n_itr = 100000
n_sim = 50
bandit_q, bandit_losses = EXP3Opt(bandit, list(bandit.nodes()), n_itr, n_sim, alpha=1)
bandit_regret = compute_regret(bandit_losses, bandit)

In [ ]:
plot_regret(bandit, [bandit_regret, upper_bound(bandit, len(bandit_regret),alpha=1)], ['EXP3', 'Upper bound'])

In [ ]:
bandit_q

independent set np hard
https://networkx.github.io/documentation/networkx-1.10/reference/algorithms.approximation.html?highlight=independent%20set#module-networkx.algorithms.approximation.independent_set

how to compute weak domination number?

regret doit être une esperance: il faut lancer plusieurs fois et faire la moyenne

### Full feedback

In [ ]:
graph_arms = [arms.ArmBernoulli(0.5), arms.ArmBernoulli(0.3), arms.ArmBernoulli(0.4), arms.ArmBernoulli(0.7), arms.ArmBernoulli(0.1)]
full_feedback = obsGraph.strong_obs_graph(5, 0, 0, graph_arms)
print(obsGraph.observability_type(full_feedback))

In [ ]:
nx.draw(full_feedback)  # networkx draw()
plt.draw()
plt.show()

In [ ]:
n_itr = 5000
n_sim = 50
full_feedback_q, full_feedback_losses = EXP3Opt(full_feedback, list(full_feedback.nodes()), n_itr, n_sim, alpha=len(full_feedback.nodes()))
full_feedback_regret = compute_regret(full_feedback_losses, full_feedback)

In [ ]:
plot_regret(full_feedback, [full_feedback_regret, upper_bound(full_feedback, len(full_feedback_regret), alpha=len(full_feedback.nodes()))], ['EXP3', 'Upper bound'])

In [ ]:
min_t = min(len(full_feedback_regret), len(bandit_regret))
plot_regret(full_feedback, [full_feedback_regret[:min_t], bandit_regret[:min_t]], ['Full feedback', 'Bandit'])

### Police officer - loopless clique

In [ ]:
graph_arms = [arms.ArmBernoulli(0.5), arms.ArmBernoulli(0.3), arms.ArmBernoulli(0.4), arms.ArmBernoulli(0.7), arms.ArmBernoulli(0.1)]
police = obsGraph.strong_obs_graph(5, 1, 0, graph_arms)
print(obsGraph.observability_type(police))

In [ ]:
nx.draw(police)  # networkx draw()
plt.draw()
plt.show()

In [ ]:
n_itr = 50000
n_sim= 50
police_q, police_losses = EXP3Opt(police, list(police.nodes()), n_itr, n_sim)
police_regret = compute_regret(police_losses, police)

In [ ]:
plot_regret(police, [police_regret, upper_bound(police, len(police_regret))], ['EXP3', 'Upper bound'])

In [ ]:
min_t = min(len(full_feedback_regret), min(len(bandit_regret),len(police_regret)))
plot_regret(police, [full_feedback_regret[:min_t], bandit_regret[:min_t], police_regret[:min_t]], ['Full feedback', 'Bandit', 'Police'])

## Weakly observable

### Revealing actions

In [ ]:
revealing = nx.DiGraph()
revealing.add_node(0, arm=arms.ArmBernoulli(0.5))
revealing.add_node(1, arm=arms.ArmBernoulli(0.3))
revealing.add_node(2, arm=arms.ArmBernoulli(0.4))
revealing.add_node(3, arm=arms.ArmBernoulli(0.7))
revealing.add_node(4, arm=arms.ArmBernoulli(0.1))
revealing.add_edge(0,0)
revealing.add_edge(0,1)
revealing.add_edge(0,2)
revealing.add_edge(0,3)
revealing.add_edge(0,4)
print(obsGraph.observability_type(revealing))

In [ ]:
nx.draw(revealing)  # networkx draw()
plt.draw()
plt.show()

In [ ]:
n_itr = 50000
n_sim = 10
revealing_q, revealing_losses = EXP3Opt(revealing, list(revealing.nodes()), n_itr, n_sim)
revealing_regret = compute_regret(revealing_losses, revealing)

In [ ]:
plot_regret(revealing, [revealing_regret, upper_bound(revealing, len(revealing_regret))], ['EXP3', 'Upper bound'])

In [ ]:
min_t = min(min(len(revealing_regret),len(full_feedback_regret)), min(len(bandit_regret),len(police_regret)))

In [ ]:
plot_regret(revealing, [full_feedback_regret[:min_t], bandit_regret[:min_t], police_regret[:min_t], revealing_regret[:min_t]], ['Full feedback', 'Bandit', 'Police','Revealing'])

## Unobservable

# Instability

## Strongly to weakly

Here we will build a strongly connected graph, run Exp3G on this graph but break the strong connectivity while the algorithm runs.
Let's start simple with 5 nodes. The graph will only contain peer-edges

In [ ]:
alpha1 = 1
beta1 = 0
n_nodes = 5
S1 = obsGraph.strong_obs_graph(n_nodes, alpha1, beta1)

In [ ]:
nx.draw(S1)  # networkx draw()
plt.draw()
plt.show()

In [ ]:
perturbations = {1000:[(0,1)], 2000:[(0,2),(1,2)]}
n_itr = 5000
n_sim = 10

obsGraph.observability_type(S1)

In [ ]:
qS1,lS1 = EXP3(S1, list(S1.nodes()), 0.5, 0.05, n_itr, n_sim)
regS1 = compute_regret(lS1, G)
dr2S1,laS1 = plot_regret(S1, [regS1], ['EXP3 on strong graph'])

W1 = S1.remove_edge(0,1)
qW1,lW1 = EXP3(W1, list(W1.nodes()), 0.5, 0.05, n_itr, n_sim)
regW1 = compute_regret(lW1, G)
dr2W1,laW1 = plot_regret(W1, [regW1], ['EXP3 on weakly connected graph'])

## Strongly to unobservable

In [ ]:
# Should transition toweakly observable at 1000th iteration, then to unobservable at 2000th iteration
perturbations = {1000:[[0,1]], 2000:[[0,2],[0,3],[0,4]]}

## Weakly to unobservable

## Bibliography

Fox & Pach, Computing the Independence Number of Intersection Graphs, math.mit.edu/~fox/paper-foxj.pdf